In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [54]:
# Import pandas
import pandas as pd

# Load the customer_data
customer_data = pd.read_csv('customer_data.csv')

# Load the app_purchases
app_purchases = pd.read_csv('inapp_purchases.csv',parse_dates=['date'])
customer_data.reg_date = pd.to_datetime(customer_data.reg_date)
# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)
# customer_data.rename(columns={'reg_date':'date'}, inplace=True)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [55]:
# Merge on the 'uid' field
uid_combined_data = app_purchases.merge(customer_data, on=['uid'], how='inner')

# Examine the results 
print(uid_combined_data.head())
print(len(uid_combined_data))

        date       uid            sku  price                  reg_date device  \
0 2017-07-10  41195147  sku_three_499    499 2017-06-26 00:00:00+00:00    and   
1 2017-07-15  41195147  sku_three_499    499 2017-06-26 00:00:00+00:00    and   
2 2017-11-12  41195147   sku_four_599    599 2017-06-26 00:00:00+00:00    and   
3 2017-09-26  91591874    sku_two_299    299 2017-01-05 00:00:00+00:00    and   
4 2017-12-01  91591874   sku_four_599    599 2017-01-05 00:00:00+00:00    and   

  gender country  age  
0      M     BRA   17  
1      M     BRA   17  
2      M     BRA   17  
3      M     TUR   17  
4      M     TUR   17  
9006


In [56]:
# Merge on the 'uid' and 'date' field
customer_data['date'] = pd.to_datetime(customer_data.reg_date.dt.date)
uid_date_combined_data = app_purchases.merge(customer_data, on=['uid', 'date'], how='inner')

# Examine the results 
print(uid_date_combined_data.head())
print(len(uid_date_combined_data))

        date       uid             sku  price                  reg_date  \
0 2016-03-30  94055095    sku_four_599    599 2016-03-30 00:00:00+00:00   
1 2015-10-28  69627745     sku_one_199    199 2015-10-28 00:00:00+00:00   
2 2017-02-02  11604973  sku_seven_1499    499 2017-02-02 00:00:00+00:00   
3 2016-06-05  22495315    sku_four_599    599 2016-06-05 00:00:00+00:00   
4 2018-02-17  51365662     sku_two_299    299 2018-02-17 00:00:00+00:00   

  device gender country  age  
0    iOS      F     BRA   16  
1    and      F     BRA   18  
2    and      F     USA   16  
3    and      F     USA   19  
4    iOS      M     TUR   16  
35


In [57]:
purchase_data = uid_combined_data.copy()

In [58]:
# Calculate the mean purchase price 
purchase_price_mean = purchase_data.price.agg('mean')

# Examine the output 
print(purchase_price_mean)

406.77259604707973


In [59]:
# Calculate the mean and median of price and age
purchase_summary = purchase_data.agg({'price': ['mean', 'median'], 'age': ['mean', 'median']})

# Examine the output 
print(purchase_summary)

             price        age
mean    406.772596  23.922274
median  299.000000  21.000000


In [60]:
# Group the data 
grouped_purchase_data = purchase_data.groupby(by=['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504  299.0  179.984378
       M       416.237308  499.0  195.001520
iOS    F       404.435330  299.0  181.524952
       M       405.272401  299.0  196.843197


In [62]:
# Compute max_purchase_date
purchase_data.reg_date  = pd.to_datetime(purchase_data.reg_date)
purchase_data.reg_date = pd.to_datetime(purchase_data.reg_date.dt.date)
current_date = pd.Timestamp('2018-03-17')
from datetime import timedelta
max_purchase_date = current_date - timedelta(days=28)

# Filter to only include users who registered before our max date
purchase_data_filt = purchase_data[purchase_data.reg_date < max_purchase_date]

# Filter to contain only purchases within the first 28 days of registration
purchase_data_filt = purchase_data_filt[(purchase_data_filt.date <= 
                        purchase_data_filt.reg_date + timedelta(days=28))]

# Output the mean price paid per purchase
print(purchase_data_filt.price.mean())

414.4237288135593


Interesting! Since our average price is 414 cents which is below $4.99 it seems that our purchasers tend towards the lower priced set of options.

In [63]:
# Set the max registration date to be one month before today
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values
month1 = np.where((purchase_data.reg_date < max_reg_date) &
                 (purchase_data.date < purchase_data.reg_date + timedelta(days=28)),
                  purchase_data.price, 
                  np.NaN)
                 
# Update the value in the DataFrame
purchase_data['month1'] = month1

# Group the data by gender and device 
purchase_data_upd = purchase_data.groupby(by=['gender', 'device'], as_index=False) 

# Aggregate the month1 and price data 
purchase_summary = purchase_data_upd.agg(
                        {'month1': ['mean', 'median'],
                        'price': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  388.204545  299.0  400.747504  299.0
1      F    iOS  432.587786  499.0  404.435330  299.0
2      M    and  413.705882  399.0  416.237308  499.0
3      M    iOS  433.313725  499.0  405.272401  299.0


Great! This value seems relatively stable over the past 28 days.

(https://strftime.org/)

In [65]:
purchase_data.to_csv('user_purchases.csv', index=False)